# neuralmt: default program

In [1]:
from default import *
import os, sys

In [2]:
cd ../

/home/bardia_az/nlp/nlpclass-1237-g-savagetokenizer/hw3


/home/bardia_az/nlp/nlpclass-1237-g-savagetokenizer/hw3/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Run the default solution on dev

In [3]:
input_path = 'data/input/dev.txt'
num = 20
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_dl = loadTestData(input_path, model.params['srcLex'],
                       device=hp.device, linesToLoad= num)
results = translate(model, test_dl)
print("\n".join(results))

20it [00:04,  4.01it/s]

i was in my first time . 
i was in and i . 
she was a woman . 
so she was to , and she was , and she was , and she was , and she was , and she was in the . 
i was i . 
a 's a first . 
i got to talk about the woman . 
i i i . 
i did n't . 
so , the the , the the , , the the , , the the , , the the , , 
i was , , " i , " i said , " 
later , later later later later later later later later later later later . 
i was in the . 
i asked to to , i was . 
i . 
she 's , , " she , , " she 's , , " she . 
but , you know , but you . 
the , the is is that it 's is , " 
what 's what is . 
that was the moment that was that the moment . 


## Evaluate the default output

In [4]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))

BLEU = 4.51 54.9/16.7/5.6/1.6 (BP = 0.471 ratio = 0.571 hyp_len = 182 ref_len = 319)


## Documentation

### The Attention Layer:

The introduction of the attention layer within autoencoders allows the model to learn which parts of the input data are most relevant for a given context, effectively enabling the model to focus its reconstruction efforts on those elements. The attention mechanism in our model, as provided in the AttentionModule class, comprises key components:

The Weighting Mechanism: The attention mechanism calculates a set of attention weights (alpha) for each element in the input sequence. These weights are learned through the W_enc and W_dec linear transformations and the V_att linear layer, making them adapt to the specific relationships between the encoder's output and the decoder's hidden state.

The Context Calculation: Once the attention weights are calculated, they are used to compute a context vector, which summarizes the most relevant information from the input sequence. In the provided implementation, the context is computed efficiently using a batched matrix multiplication, which significantly improves computational performance.

As was mentioned in the documentation, attention is defined as follows:
\[\mathrm{score}_i = W_{enc}( h^{enc}_i ) + W_{dec}( h^{dec} )\]
Define the $\alpha$ vector as follows:

\[\alpha = \mathrm{softmax}(V_{att} \mathrm{tanh} (\mathrm{score}))\]
The we define the context vector using the $\alpha$ weights for each source side index $i$:

\[c = \sum_i \alpha_i \times h^{enc}_i\]

### Effect of Attention Mechanism:

The introduction of the attention mechanism into the autoencoder model has a profound impact on its performance. This can be observed through the significant increase in the BLEU score from 1.8637 to 14.2469, indicating the substantial improvement in the model's ability to capture and reconstruct input sequences.

The attention mechanism allows the model to focus on the most critical elements of the input data during the decoding process, resulting in improved reconstruction quality. This is especially beneficial for tasks where certain elements in the input sequence are more important than others, such as machine translation or text summarization.

#### here is the implementation:


In [5]:
class AttentionModule(nn.Module):
    def __init__(self, attention_dim):
        """
        You shouldn't deleted/change any of the following defs, they are
        essential for successfully loading the saved model.
        """
        super(AttentionModule, self).__init__()
        self.W_enc = nn.Linear(attention_dim, attention_dim, bias=False)
        self.W_dec = nn.Linear(attention_dim, attention_dim, bias=False)
        self.V_att = nn.Linear(attention_dim, 1, bias=False)
        self.softmax = nn.Softmax(dim = 0)

    # Start working from here, both 'calcAlpha' and 'forward' need to be fixed
    def calcAlpha(self, decoder_hidden, encoder_out):
        """
        param encoder_out: (seq, batch, dim),
        param decoder_hidden: (seq, batch, dim)
        """
        enc = self.W_enc( encoder_out )
        dec = self.W_dec( decoder_hidden )
        scores = enc + dec
        beta = self.V_att( torch.nn.functional.tanh( scores ) )
        alpha = self.softmax( beta )
        return alpha

    def forward(self, decoder_hidden, encoder_out):
        """
        encoder_out: (seq, batch, dim),
        decoder_hidden: (seq, batch, dim)
        """
        alpha = self.calcAlpha(decoder_hidden, encoder_out) # seq, batch, dim=1
        context = torch.sum(alpha * encoder_out, dim=0).unsqueeze(0)
        return context, alpha.permute(2, 1, 0)

## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?

We tried Ensembling and Beam Searching:

### Ensembling
We load all the check point files existing in the directory that --model command line option point to, and create a list of Seq2Seq models. Then we run the translation on all the models and obtain the scores of each token in the destination vocabulary for the current word. First we tried to add those scores for all the instances of Seq2Seq models and in the end, choose the maximum value. However, it worsened the results. Then, we decided to choose the maximum-score (best) token for each of the models, and then select the token that has been chosen with most of the models. In this case, the results were better but still worse than the baseline. So, we couldn't succeed in Ensembling.
Note that, the --model command line option now can point to a checkpoint file or a directory. In the first case, only the selected checkpoint file is loaded and of course no ensembling happens. In the second case, the code loads all the checkpoint files available in the directory and runs ensembling inference.

### Beam Searching